In [1]:
import sys
sys.path.append('../utils')
import fvdb.nn as fvnn
import torch 
import igl
from diffusion_tensor import *
import mesh_tools as mt
import matplotlib.pyplot as plt
from fvdb_utils import show_vdb_marching_cubes


In [2]:
tens = torch.load('../../output/acropolis/gen_0_4.pt', weights_only=False)

In [3]:
ind=2
disp_d = DiffusionTensor(tens.grid[ind], tens.feature[ind]).remove_mask()
disp_d.get_global().colored_PC(.07)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000225…

### Test direct MC

In [188]:
train_grid = disp_d
train_grid.feature.jdata[:, 3:6] = torch.clip(train_grid.feature.jdata[:, 3:6], -.5, .5)

In [189]:
dual_grid = train_grid.grid.dual_grid()
dual_centers = dual_grid.grid_to_world(dual_grid.ijk.float())
normals, vstars, _, mask = DiffusionTensor.get_feature_data(train_grid.get_global().jdata)
# splat vstars + normals on dual grid
print(vstars.shape)
features = torch.cat((normals, (vstars*normals).sum(-1, True), torch.ones_like(vstars[:, :1])), -1)
new_feature = dual_grid.splat_trilinear(train_grid.grid.jagged_like(vstars), train_grid.grid.jagged_like(features))

nfdata = new_feature.jdata[:, :-1]/new_feature.jdata[:, -1:]
dual_sign = (dual_centers.jdata*nfdata[:,:3]).sum(-1, True)-nfdata[:,3:]
dual_sign = 2.*(dual_sign>0)-1
dual_tens = fvnn.VDBTensor(dual_grid, dual_grid.jagged_like(dual_sign[:, None]))

torch.Size([118504, 3])


In [190]:
dual_data = dual_tens.to_dense().cpu().detach().numpy()
primal_data = train_grid.to_dense().cpu().detach().numpy()

mt.plotSlice(dual_data[0, ..., 0], .01)

interactive(children=(IntSlider(value=114, description='s', max=228), Output()), _dom_classes=('widget-interac…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [191]:
show_vdb_marching_cubes(dual_tens)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

### Robust hierarchy

In [109]:
def get_pool(self, k_s=2):
    '''make sure it's global first'''
    feature, grid = self.grid.avg_pool(k_s, self.feature, k_s)
    feature.jdata /= feature.jdata[:, -1:].clone()
    return  DiffusionTensor(grid, feature)


In [110]:
global_fine_grid = disp_d.get_global()

first_ks = 128
primal_data = get_pool(global_fine_grid, first_ks)

if not primal_data.to_dense()[..., -1].sum()==8:
    print('WARNING: ASSUMPTION BROKEN')
occupancy_tensor = fvnn.VDBTensor(primal_data.grid, primal_data.grid.jagged_like(torch.zeros(8, device='cuda')[:, None]))
# -1: INSIDE, 0: ACTIVE, 1: OUTSIDE

In [111]:
primal_data.grid.ijk.jdata

tensor([[-1, -1, -1],
        [-1, -1,  0],
        [-1,  0, -1],
        [-1,  0,  0],
        [ 0, -1, -1],
        [ 0, -1,  0],
        [ 0,  0, -1],
        [ 0,  0,  0]], device='cuda:0', dtype=torch.int32)

In [132]:
### KEEEP

# see which are in the next level
K=2
current_ks=first_ks//K
primal_data = get_pool(global_fine_grid, current_ks)
normals, vstars, _, _ = DiffusionTensor.get_feature_data(primal_data.jdata)

sub_features, sub_grid = occupancy_tensor.grid.subdivide(K, occupancy_tensor.feature)
sub_centers = sub_grid.grid_to_world(sub_grid.ijk.float())

is_void = (primal_data.grid.ijk_to_index(sub_grid.ijk).jdata == -1)[:, None] # subdivided voxel not in grid

to_decide_sign = ((sub_features==0)*(is_void)).jdata # --> Decide sign
# else keep value 

# --> Decide sign
points_neigh_idx = primal_data.grid.neighbor_indexes(sub_grid.ijk, 2).jdata
mask_exists = (points_neigh_idx!=-1)*1.

local_delta = (sub_centers.jdata.unsqueeze(1).unsqueeze(1).unsqueeze(1)-vstars[points_neigh_idx])
local_sdf = (local_delta*normals[points_neigh_idx]).sum(-1)
local_sdf[torch.logical_not(mask_exists)] = 0
local_dist = (local_delta**2).sum(-1)
local_dist[torch.logical_not(mask_exists)] = 1e20

predicted_sdf = (torch.softmax(-10000*local_dist.view(len(is_void), -1), -1)*local_sdf.view(len(is_void), -1))
predicted_sdf = predicted_sdf.sum(-1, True)

# sub_features.jdata[to_decide_sign] = 2.*(predicted_sdf[to_decide_sign]>0)-1
sub_features.jdata[to_decide_sign] = predicted_sdf[to_decide_sign]

# prune
sub_neigh_idx = sub_grid.neighbor_indexes(sub_grid.ijk, 1).jdata

# TODO

occupancy_tensor = fvnn.VDBTensor(sub_grid, sub_features)

first_ks = current_ks

### Occupancy tensor: contains atoccupancy_tensor.grid.subdivide(2, occupancy_tensor.feature) each level, +1 for inside voxels, -1 for outside voxels, 0 for voxel containing the surface

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.93 GiB. GPU 0 has a total capacity of 9.75 GiB of which 2.57 GiB is free. Including non-PyTorch memory, this process has 7.08 GiB memory in use. Of the allocated memory 6.65 GiB is allocated by PyTorch, and 186.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [129]:
predicted_sdf

tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], device='cuda:0')

In [130]:
# ### KEEEP
# # prune
# sub_neigh_idx = sub_grid.neighbor_indexes(sub_grid.ijk, 1).jdata

# subdivided_occupancy_grid = fvnn.VDBTensor(sub_grid, sub_features)
# # subdivided_occupancy_grid = fvnn.VDBTensor(sub_grid, sub_grid.jagged_like(p_sdf))

In [133]:
mt.plotSlice(primal_data.to_dense()[0, ..., -1].cpu().detach().numpy(), 1.)

interactive(children=(IntSlider(value=57, description='s', max=113), Output()), _dom_classes=('widget-interact…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>

In [108]:
mt.plotSlice(occupancy_tensor.to_dense()[0, ..., 0].cpu().detach().numpy(), .1)

interactive(children=(IntSlider(value=16, description='s', max=31), Output()), _dom_classes=('widget-interact'…

<function mesh_tools.plotSlice.<locals>.<lambda>(s)>